# Context
While many public datasets (on Kaggle and the like) provide Apple App Store data, there are not many counterpart datasets available for Google Play Store apps anywhere on the web. On digging deeper, I found out that iTunes App Store page deploys a nicely indexed appendix-like structure to allow for simple and easy web scraping. On the other hand, Google Play Store uses sophisticated modern-day techniques (like dynamic page load) using JQuery making scraping more challenging.
Content
Each app (row) has values for catergory, rating, size, and more.

1. App: Application name
2. Category: Category the app belongs to
3. Rating: Overall user rating of the app (as when scraped)
4. Reviews: Number of user reviews for the app (as when scraped)
5. Size: Size of the app (as when scraped)
6. Installs: Number of user downloads/installs for the app (as when scraped)
7. Type: Paid or Free
8. Price: Price of the app (as when scraped)
9. Content Rating: Age group the app is targeted at - Children / Mature 21+ / Adult
10.Genres: An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to Music, Game, Family genres.
11. Last Updated: Date when the app was last updated on Play Store (as when scraped)
12. Current Ver: Current version of the app available on Play Store (as when scraped)
13. Android Ver: Min required Android version (as when scraped)

In [1]:
import math
import pdb
import pandas as pd
import numpy as np
import datetime
import re
import seaborn as sns
from scipy import stats
from scipy.stats import t, sem
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt

In [2]:
# Hide deprecated warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# data 1 and data 2 were merged (inner join) 
#so that only the apps that had reviews lie in the dataset

In [4]:
# Load data from the folder where I uploaded the csv file
data_merged=pd.read_csv("/Users/AirMorena/Desktop/final_proj/csv/data_merged.csv")

In [5]:
data_merged.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,NaN,NaN,NaN,NaN
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000


In [6]:
print("Shape of data (samples, features): ",data_merged.shape)

Shape of data (samples, features):  (122662, 17)


In [7]:
print("Data Types: \n", data_merged.dtypes.value_counts())

Data Types: 
 object     13
float64     3
int64       1
dtype: int64


In [8]:
#Returns the column names.
data_merged.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity',
       'Sentiment_Subjectivity'],
      dtype='object')

In [9]:
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122662 entries, 0 to 122661
Data columns (total 17 columns):
App                       122662 non-null object
Category                  122662 non-null object
Rating                    122622 non-null float64
Reviews                   122662 non-null int64
Size                      122662 non-null object
Installs                  122662 non-null object
Type                      122662 non-null object
Price                     122662 non-null object
Content Rating            122662 non-null object
Genres                    122662 non-null object
Last Updated              122662 non-null object
Current Ver               122662 non-null object
Android Ver               122662 non-null object
Translated_Review         72605 non-null object
Sentiment                 72615 non-null object
Sentiment_Polarity        72615 non-null float64
Sentiment_Subjectivity    72615 non-null float64
dtypes: float64(3), int64(1), object(13)
memory usage: 15.

In [10]:
data_merged.isnull().sum()

App                           0
Category                      0
Rating                       40
Reviews                       0
Size                          0
Installs                      0
Type                          0
Price                         0
Content Rating                0
Genres                        0
Last Updated                  0
Current Ver                   0
Android Ver                   0
Translated_Review         50057
Sentiment                 50047
Sentiment_Polarity        50047
Sentiment_Subjectivity    50047
dtype: int64

In [11]:
# we need to convert selected variables from object to float or int

In [12]:
# Finding out the number of NaN values in each column
data_merged.isnull().sum().sort_values(ascending=False)

Translated_Review         50057
Sentiment_Subjectivity    50047
Sentiment                 50047
Sentiment_Polarity        50047
Rating                       40
Type                          0
Category                      0
Reviews                       0
Size                          0
Installs                      0
Content Rating                0
Price                         0
Genres                        0
Last Updated                  0
Current Ver                   0
Android Ver                   0
App                           0
dtype: int64

In [13]:
##############################################################################################

In [14]:
# DATA CLEANING

In [15]:
# 1) convert features that are object that should be numeric into int or float

In [16]:
# Installs

In [17]:
# Feature Installs should be interger, need to remove the + and coma

In [18]:
data_merged['Installs'].unique()

array(['500,000+', '1,000,000+', '100,000+', '5,000,000+', '10,000+',
       '10,000,000+', '100,000,000+', '1,000,000,000+', '50,000,000+',
       '500,000,000+', '1,000+', '50,000+', '5,000+'], dtype=object)

In [ ]:
# convert installs to numeric int: replace+ by '', but it leaves a coma
#data_merged['Installs'].str.replace('+','')

In [ ]:
# replace comma by nothing and convert to int,and save it into data_merged['Installs'] 
data_merged['Installs']=data_merged['Installs'].str.replace('+','').str.replace(',','').astype(int)

In [ ]:
# check if it worked
data_merged['Installs'].unique()

array([    500000,    1000000,     100000,    5000000,      10000,
         10000000,  100000000, 1000000000,   50000000,  500000000,
             1000,      50000,       5000])

In [ ]:
####Feature Size

In [ ]:
data_merged['Size'].unique()

array(['14M', '33M', '37M', '39M', '12M', '25M', '6.1M', '11M',
       'Varies with device', '15M', '24M', '17M', '2.4M', '27M', '16M',
       '2.9M', '38M', '21M', '52M', '8.2M', '4.2M', '2.8M', '5.0M',
       '5.9M', '19M', '73M', '6.8M', '4.0M', '10M', '2.1M', '42M', '30M',
       '29M', '3.9M', '18M', '3.2M', '20M', '26M', '48M', '22M', '8.5M',
       '28M', '46M', '23M', '7.1M', '40M', '7.8M', '6.6M', '5.1M', '79k',
       '32M', '2.2M', '4.7M', '2.7M', '13M', '118k', '44M', '7.3M',
       '695k', '6.2M', '56M', '3.1M', '31M', '8.0M', '7.9M', '1.4M',
       '7.2M', '3.8M', '41M', '9.0M', '4.9M', '8.1M', '3.7M', '4.5M',
       '2.6M', '6.9M', '7.4M', '1.9M', '1.8M', '2.3M', '5.2M', '1.2M',
       '59M', '5.6M', '72M', '9.6M', '43M', '3.6M', '9.5M', '8.9M', '65M',
       '79M', '8.4M', '58M', '50M', '45M', '53M', '68M', '66M', '35M',
       '76M', '9.4M', '4.3M', '67M', '60M', '5.5M', '3.3M', '7.5M',
       '9.1M', '8.3M', '9.2M', '7.0M', '77M', '5.7M', '5.3M', '232k',
       '99M',

In [ ]:
data_merged['Size'].isna().count()

122662

In [ ]:
data_merged['Size'].dtypes

dtype('O')

In [ ]:
# in order to know how to replace the NaN
# we need to look at the distribution of the non nulls to decide how to deal
# with the nan, replace by zero or by the mean/median?

In [ ]:
## for loop to iterate over Size:

# create a new empty list:
my_new_size=[]

# create a list from series data['Size']
sizelist=data_merged['Size'].tolist()
                     
for size in sizelist:# size element in the list
       
        if size=='Varies with device':
            my_new_size.append(size)
            
        else:
            print(size)
            print(size[1:0])
            unit=size[-1]# the last character of the string is -1
            value=float(size[:-1])#change the value to float for all except last character

            if unit=='M':
                    new_value=value*1000000
                    my_new_size.append(new_value)
            elif unit=='k':
                    new_value=value/1000
                    my_new_size.append(new_value)




14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M

33M



19M

19M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

73M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

6.8M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

4.0M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M




10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

10M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M

22M


6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

2.2M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M

4.7M


24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

24M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M

23M


...


8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

8.1M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

1

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

7.5M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

5.8M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

8.5M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

7.6M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M

6.1M


7.4M

7.4M

7.4M

7.4M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

3.0M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

14M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

38M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

3.9M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M

5.4M


4.3M

4.3M

4.3M

4.3M

4.3M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

25M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

7.1M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

9.2M

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

720k

5.7M

5.7M

5.7M

5.7M

5.7M

5.7M

5.7M

5.7M

5.


13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

13M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

8.2M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M

27M


In [ ]:
 print(my_new_size)

In [ ]:
data_merged['Size']= my_new_size

In [ ]:
data_merged['Size'].unique()

In [ ]:
# condition data[Size] without Varies with device: 
condition1=data_merged[data_merged['Size']!='Varies with device']['Size']

In [ ]:
condition1.unique()

In [ ]:
#vizualisation distribution Size values excluding Varies with device
plt.figure(figsize=(12,8))
_=sns.boxplot(condition1.astype(float), color='b')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,8))
_=sns.distplot(condition1.astype(float), color='b')
plt.tight_layout()

In [ ]:
condition1.astype(float).describe()

In [ ]:
# still need to convert varies with device with 0 or mean?

In [ ]:
# still need to convert size to float

In [ ]:
data_merged.head()

In [ ]:
##### Feature Price 

In [ ]:
#Price is store as an object
#data_merged['Price'].dtypes

In [ ]:
data_merged['Price'].unique()

In [ ]:
# remove dollar from Price
data_merged['Price']= data_merged['Price'].str.replace("$","")

In [ ]:
data_merged['Price'].unique()

In [ ]:
data_merged['Price'] = data_merged['Price'].astype(float)

In [ ]:
data_merged['Price'].dtypes

In [ ]:
sns.boxplot(data_merged['Price'])
plt.grid(True)
plt.tight_layout()

In [ ]:
# mean price is 10 dollars and median 6.99 among the apps that have a price
data_merged['Price'].describe()

In [ ]:
# Reviews
data_merged['Reviews'].unique()

In [ ]:
len(data_merged['Reviews'].unique())

In [ ]:
data_merged['Reviews'].dtypes

In [ ]:
data_merged['Reviews'].describe()

In [ ]:
print("range number of reviews: ", data_merged['Reviews'].min(),"-",data_merged['Reviews'].max())

In [ ]:
_=plt.figure(figsize=(12,8))
sns.distplot(data_merged['Reviews'], bins=10 )

In [ ]:
sns.boxplot(data_merged['Reviews'])
plt.grid(True)
plt.tight_layout()

In [ ]:
# vizualise, once apply log to reviews
# or create groups

In [ ]:
# Feature Rating: already a float , range should be within 0 to 5
# already a float , range should be within 0 to 5

In [ ]:
print("range:", data_merged['Rating'].min(),"-",data_merged['Rating'].max())

In [ ]:
print(data_merged['Rating'].isna().sum(),"missing values out of", len(data_merged['Rating']))

In [ ]:
# replace nan with mean rating
data_merged.fillna(data_merged['Rating'].mean(),inplace= True)
data_merged.isna().sum()

In [ ]:
#select those rows that have a null value in the column share
isnull_condition = (data_merged['Rating'].isnull())

In [ ]:
non_null_rating = data_merged[~isnull_condition]['Rating']
#vizualisation distribution 0 of share values
_=sns.distplot(non_null_rating)
plt.title("Apps average rating")
plt.grid(True)
plt.tight_layout()

In [ ]:
# current version

In [ ]:
#data_merged['Current Ver'].unique()

In [ ]:
data_merged['Current Ver'].isna().sum()

In [ ]:
temp=data_merged['Current Ver'].replace(np.nan,'Varies with device')

In [ ]:
temp=temp.apply(lambda x: 'Varies with device' if x=='Varies with device'  else  re.findall('^[0-9]\.[0-9]|[\d]|\W*',str(x))[0] )

In [ ]:
temp.unique()

In [ ]:
temp

In [ ]:
data_merged['Current_Ver_updated']=temp

In [ ]:
data_merged.head()

In [ ]:
data_merged.info()

In [ ]:
data_merged['Last Updated'].unique()

In [ ]:
# extract month an year 

In [ ]:
# export data_clean
# Export data to my chosen folder
#data_merged.to_csv("/Users/AirMorena/Desktop/final_proj/csv/data_clean.csv", index=False, encoding='utf-8')